In [ ]:
from pathlib import Path

import polars as pl
import yaml

from nhts.trips import preprocess

In [53]:
hhs = {}

years = [2022, 2017, 2009, 2001]
names = ["hhv2pub.csv", "hhpub.csv", "HHV2PUB.CSV", "HHPUB.csv"]

with open("nhts/hh_dictionary.yaml") as f:
    config = yaml.safe_load(f)

column_mapping = config["column_mappings"]


# check table keys

for col in column_mapping.keys():
    col_sets = []
    for year, name in zip(years, names):
        if col not in config:
            continue  # skip non categorical columns

        path = Path("~/Data/foundata/NHTS") / str(year) / name
        data = pl.read_csv(path, ignore_errors=True)

        # select columns
        if col not in data.columns:
            print(f"column {col} not in year {year}")
            continue

    for i in range(1, len(col_sets)):
        if col_sets[0] != col_sets[i]:
            print(f"mismatch in column {col}:")
            print(f" 2022: {col_sets[0]} vs {years[i]}:  {col_sets[i]}")


for year, name in [
    (2022, "hhv2pub.csv"),
    (2017, "hhpub.csv"),
    (2009, "HHV2PUB.CSV"),
    (2001, "HHPUB.csv"),
]:
    path = Path("~/Data/foundata/NHTS") / str(year) / name
    data = pl.read_csv(path, ignore_errors=True)

    # select columns
    available_cols = set(data.columns)
    required_cols = set(column_mapping.keys())
    missing_cols = required_cols - available_cols
    if missing_cols:
        print(f"year {year} is missing columns: {missing_cols}")
    column_mapping_filtered = {
        k: v for k, v in column_mapping.items() if k in available_cols
    }
    cols = list(column_mapping_filtered.keys())
    data = data.select(cols)

    # apply mappings
    data = data.with_columns(
        pl.col(col)
        .replace_strict(config[col].get(year, config[col]["default"]), default=None)
        .fill_null(pl.col(col))
        for col in column_mapping_filtered.keys()
        if col in config
    )

    # rename columns
    data = data.rename(column_mapping_filtered)

    # split year and month
    data = data.with_columns(
        data["date"].cast(pl.String).str.slice(0, 4).cast(pl.Int32).alias("year"),
        data["date"].cast(pl.String).str.slice(4).cast(pl.Int32).alias("month"),
    )

    # ensure unique household ids across years
    data = data.with_columns((pl.col("hid") + year * 10_000_000_000).alias("hid"))

    hhs[year] = data

    print(f"{year}: {len(hhs[year])} hhs found")

column HH_RACE not in year 2001
year 2022 is missing columns: {'HHR_RACE'}
2022: 7893 hhs found
year 2017 is missing columns: {'HHR_RACE'}
2017: 129696 hhs found
year 2009 is missing columns: {'HHR_RACE'}
2009: 150147 hhs found
year 2001 is missing columns: {'HH_RACE'}
2001: 69817 hhs found


In [58]:
persons = {}

years = [2022, 2017, 2009, 2001]
names = ["perv2pub.csv", "perpub.csv", "PERV2PUB.CSV", "PERPUB.csv"]

with open("nhts/person_dictionary.yaml") as f:
    config = yaml.safe_load(f)

column_mapping = config["column_mappings"]


# check table keys

for col in column_mapping.keys():
    col_sets = []
    for year, name in zip(years, names):
        if col not in config:
            continue  # skip non categorical columns

        path = Path("~/Data/foundata/NHTS") / str(year) / name
        data = pl.read_csv(path, ignore_errors=True)

        # select columns
        if col not in data.columns:
            print(f"column {col} not in year {year}")
            continue
        col_sets.append(set(data[col].unique().to_list()))

    for i in range(1, len(col_sets)):
        if col_sets[0] != col_sets[i]:
            print(f"mismatch in column {col}:")
            print(f" 2022: {col_sets[0]} vs {years[i]}:  {col_sets[i]}")


for year, name in zip(years, names):
    path = Path("~/Data/foundata/NHTS") / str(year) / name
    data = pl.read_csv(path, ignore_errors=True)

    # select columns
    data = data.select(column_mapping.keys())

    # apply mappings
    data = data.with_columns(
        pl.col(col)
        .replace_strict(config[col].get(year, config[col]["default"]), default=None)
        .fill_null(pl.col(col))
        for col in column_mapping.keys()
        if col in config
    )

    # rename columns
    data = data.rename(column_mapping)

    # ensure unique person ids across years
    data = data.with_columns(
        ((pl.col("hid") + year * 10_000_000_000) * 10 + pl.col("phid")).alias("pid")
    )

    persons[year] = data

    print(f"{year}: {len(persons[year])} persons found")

mismatch in column EDUC:
 2022: {1, 2, 3, 4, 5, 6, 7, 8, -1} vs 2017:  {1, 2, 3, 4, 5, -8, -7, -1}
mismatch in column EDUC:
 2022: {1, 2, 3, 4, 5, 6, 7, 8, -1} vs 2009:  {1, 2, 3, 4, 5, -9, -8, -7, -1}
mismatch in column EDUC:
 2022: {1, 2, 3, 4, 5, 6, 7, 8, -1} vs 2001:  {1, 2, 3, 4, 5, 6, 7, 8, -9, -8, -7, -1}
mismatch in column MEDCOND:
 2022: {1, 2, -9, -8, -7} vs 2009:  {1, 2, -9, -8, -7, -1}
mismatch in column MEDCOND:
 2022: {1, 2, -9, -8, -7} vs 2001:  {1, 2, -9, -8, -7, -1}
mismatch in column PRMACT:
 2022: {1, 2, 3, 4, 5, 97, -1} vs 2017:  {1, 2, 3, 4, 5, 6, 97, -8, -7, -1}
mismatch in column PRMACT:
 2022: {1, 2, 3, 4, 5, 97, -1} vs 2009:  {1, 2, 3, 4, 5, 6, 7, -9, -8, -7, -1}
mismatch in column PRMACT:
 2022: {1, 2, 3, 4, 5, 97, -1} vs 2001:  {1, 2, 3, 4, 5, 6, 7, -9, -8, -7, -1}
mismatch in column R_RELAT:
 2022: {1, 2, 3, 4, 5, 6, 7, -9} vs 2017:  {1, 2, 3, 4, 5, 6, 7, -9, -8, -7}
mismatch in column R_RELAT:
 2022: {1, 2, 3, 4, 5, 6, 7, -9} vs 2009:  {1, 2, 3, 4, 5, 6, 7,

In [59]:
persons[2022].head()

phid,hid,age,education,disability,employment_status,relationship_to_respondent,sex,has_license,pid
i64,i64,i64,str,str,str,str,str,str,i64
1,9000013002,39,"""graduate_degree""","""no""","""unemployed""","""self""","""female""","""yes""",202290000130021
2,9000013002,42,"""graduate_degree""","""no""","""employed""","""spouse/partner""","""male""","""yes""",202290000130022
3,9000013002,8,"""unknown""","""no""","""employed""","""child""","""unknown""","""unknown""",202290000130023
4,9000013002,5,"""unknown""","""no""","""employed""","""child""","""unknown""","""unknown""",202290000130024
1,9000013016,32,"""bachelors_degree""","""yes""","""employed""","""self""","""female""","""yes""",202290000130161


In [18]:
pops = {}

with open("nhts/trip_dictionary.yaml") as f:
    config = yaml.safe_load(f)

for year, name in [
    (2022, "tripv2pub.csv"),
    (2017, "trippub.csv"),
    (2009, "DAYV2PUB.CSV"),
    (2001, "DAYPUB.csv"),
]:
    path = Path("~/Data/foundata/NHTS") / str(year) / name
    trips = pl.read_csv(path, ignore_errors=True)
    trips = preprocess(trips, year=year, config=config)
    print(len(trips))
    # pam_trips = read.load_travel_diary(
    #     trips=trips.to_pandas()
    # )
    # pops[year] = pam_trips

30844
915050
1060185
472704


In [9]:
root = Path("~/Data/foundata/NHTS/2017")
codesheet = pl.read_excel(root / "codebook_v1.2.xlsx", sheet_name="CODEBOOK_HH")

codesheet = (
    codesheet.with_columns(
        [
            pl.when(pl.col(c) == "").then(None).otherwise(pl.col(c)).alias(c)
            for c in codesheet.columns
            if codesheet[c].dtype == pl.String
        ]
    )
    .fill_null(strategy="forward")
    .filter(pl.col("Type") == "C")
    .with_columns(
        [
            pl.col("Code / Range")
            .str.splitn("=", 2)
            .struct.field("field_0")
            .alias("key"),
            pl.col("Code / Range")
            .str.splitn("=", 2)
            .struct.field("field_1")
            .alias("value"),
        ]
    )
    .filter(pl.col("key").cast(pl.Int64, strict=False).is_not_null())
    .with_columns(key=pl.col("key").cast(pl.Int32))
)

mapper = {}
for i, frame in codesheet.group_by("Name"):
    column = str(i[0])
    mapper[column] = {}
    keys = frame.select(pl.col("key")).rows()
    values = frame.select(pl.col("value")).rows()
    for k, v in zip(keys, values):
        mapper[column][int(k[0])] = str(v[0])

mapper

{'BIKE2SAVE': {-9: 'Not ascertained',
  -8: "I don't know",
  -7: 'I prefer not to answer',
  1: 'Strongly agree',
  2: 'Agree',
  3: 'Neither Agree or Disagree',
  4: 'Disagree',
  5: 'Strongly disagree'},
 'HBHTNRNT': {-9: 'Not ascertained',
  0: '0-4%',
  5: '5-14%',
  20: '15-24%',
  30: '25-34%',
  40: '35-44%',
  50: '45-54%',
  60: '55-64%',
  70: '65-74%',
  80: '75-84%',
  90: '85-94%',
  95: '95-100%'},
 'HHRELATD': {1: 'Yes', 2: 'No'},
 'CAR': {-8: "I don't know",
  -7: 'I prefer not to answer',
  -1: 'Appropriate skip',
  1: 'Daily',
  2: 'A few times a week',
  3: 'A few times a month',
  4: 'A few times a year',
  5: 'Never'},
 'MSASIZE': {1: 'In an MSA of Less than 250,000',
  2: 'In an MSA of 250,000 - 499,999',
  3: 'In an MSA of 500,000 - 999,999',
  4: 'In an MSA or CMSA of 1,000,000 - 2,999,999',
  5: 'In an MSA or CMSA of 3 million or more',
  6: 'Not in MSA or CMSA'},
 'HHRESP': {1: 'None', 2: 'None', 3: 'None', 4: 'None', 5: 'None', 7: 'None'},
 'SPHONE': {-9: 'N